# Re-projecting polder data
---
* from [EPSG:7415](https://spatialreference.org/ref/epsg/7415/) (Amersfoort / RD New + NAP height)
* to [EPSG:3395](https://spatialreference.org/ref/epsg/3395/) (WGS 84 / World Mercator)

In [7]:
from pathlib import Path
import pandas as pd
import geopandas as gpd

### Read data

In [ ]:
dbfs = list(Path('../data').glob('*.dbf'))
include = ['LABCODE', 'SAMPLINGCO', 'X_COORDINA', 'Y_COORDINA', 'PAIR_LENGT']
read_dbf = lambda f: gpd.read_file(f, include_fields=include)
data = pd.concat(
    (read_dbf(f) for f in dbfs),
    keys=(f.stem for f in dbfs),
    names=['dbf', 'idx'],
).reset_index()

### Re-project

In [ ]:
geometry = gpd.GeoSeries.from_xy(data.X_COORDINA, data.Y_COORDINA)
gdf = (
    gpd.GeoDataFrame(
        data,
        geometry = geometry,
        crs = 'EPSG:7415',
    )
    .to_crs('EPSG:3395')
    .assign(
        longitude = lambda gdf: gdf.geometry.x,
        latitude = lambda gdf: gdf.geometry.y,
    )
)
gdf.to_excel('../output/reprojected_polder_data.xlsx')

In [ ]:
gdf

### Explore

In [ ]:
gdf.explore()

In [ ]:
gdf.explore().save('../index.html')